In [8]:
import numpy as np
from scipy.optimize import minimize, minimize_scalar

# Parameters
beta = 0.96  # discount factor
eta = 2.0    # coefficient of relative risk aversion
gamma = 0.5  # disutility from working
alpha = 0.33 # capital share in production
delta = 0.1  # depreciation rate
tau = 0.2    # tax rate
psi = 0.1    # public infrastructure elasticity
nu = 0.6     # debt-to-GDP ratio limit
omega = 0.1  # parent education time factor
phi_p = 0.5  # share of parental input in education
psi_e = 0.5  # CES parameter for education inputs
theta_I = 0.7  # human capital investment effectiveness

# Government parameters
sigma_bar = 0.2  # public investment inefficiency
G = 0.1  # grants (as a fraction of GDP)

# Grids
k_grid = np.linspace(0, 20, 100)
h_grid = np.linspace(1, 10, 50)
Z_grid = np.linspace(1, 10, 20)

# Number of periods
T = 60  # Total periods
working_periods = 37
parent_periods = 18

def utility(c, l):
    return ((c * (1 - l)**gamma)**(1 - eta) - 1) / (1 - eta)

def production(K, H, L, Z):
    return Z**psi * K**alpha * (H * L)**(1 - alpha)

def wage(K, H, L, Z):
    return H * (1 - alpha) * Z**psi * (K / (H * L))**alpha

def interest_rate(K, H, L, Z):
    return alpha * Z**psi * (H * L / K)**(1 - alpha) - delta

def human_capital_investment(e, I_e, h):
    I_c = (phi_p * (e)**psi_e + (1 - phi_p) * (I_e / 18)**psi_e)**(1 / psi_e)
    return I_c**theta_I * h**(1 - theta_I)

def bellman_operator(V, s, K, H, L, Z, I_e):
    V_next = np.zeros_like(V)
    
    # Calculate wage and interest rate once for this iteration
    w = wage(K, H, L, Z)
    r = interest_rate(K, H, L, Z)
    
    for i, k in enumerate(k_grid):
        for j, h in enumerate(h_grid):
            if s >= working_periods:
                # Retired
                def objective(c):
                    if c <= 0:
                        return np.inf
                    
                    k_next = (1 + r - delta) * k - (1 + tau) * c + T
                    if k_next < k_grid[0] or k_next > k_grid[-1]:
                        return np.inf
                    
                    k_next_index = np.searchsorted(k_grid, k_next)
                    V_interp = np.interp(k_next, k_grid, V[:, j]) if s < T-1 else 0
                    
                    return -(utility(c, 1) + beta * V_interp)
                
                result = minimize_scalar(objective, bounds=(0, k + T), method='bounded')
                V_next[i, j] = -result.fun
            
            else:
                # Working
                def objective(c_l):
                    c, l = c_l[0], c_l[1]
                    if c <= 0 or l < 0 or l > 1:
                        return np.inf
                    
                    if s <= parent_periods:
                        # Working parent
                        e = omega * np.log(h) * (1 - l) / (1 + omega * np.log(h))
                        h_child_next = human_capital_investment(e, I_e, 1 if s == 1 else h)
                    else:
                        e = 0
                        h_child_next = h
                    
                    k_next = (1 - tau) * w * h * l + (1 + r - delta) * k - (1 + tau) * c + T
                    if k_next < k_grid[0] or k_next > k_grid[-1]:
                        return np.inf
                    
                    k_next_index = np.searchsorted(k_grid, k_next)
                    h_next_index = np.searchsorted(h_grid, h_child_next)
                    V_interp = np.interp(k_next, k_grid, V[:, h_next_index]) if s < T-1 else 0
                    
                    return -(utility(c, 1 - l) + beta * V_interp)
                
                result = minimize(objective, [k/2, 0.5], bounds=((0, k + w * h), (0, 1)), method='L-BFGS-B')
                V_next[i, j] = -result.fun
    
    return V_next

def solve_household_problem(K, H, L, Z, I_e):
    V = np.zeros((len(k_grid), len(h_grid), T))
    
    for t in range(T-1, -1, -1):
        V[:, :, t] = bellman_operator(V[:, :, t], t, K, H, L, Z, I_e)
    
    return V

def government_budget(Y, K, H, L, Z, I_e, T, D_prev, r_prev):
    tax_revenue = tau * (wage(K, H, L, Z) * H * L + (r_prev - delta) * K)
    D = r_prev * D_prev + I_e + (Z - (1 - delta) * Z) / (1 - sigma_bar) + T - G * Y - tax_revenue
    return D


def golden_section_search(f, a, b, tol=1e-5):
    gr = (np.sqrt(5) + 1) / 2  # Golden ratio
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(c - d) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b - a) / gr
        d = a + (b - a) / gr
    return (b + a) / 2

def solve_steady_state(max_iter=100, tol=1e-4):
    # Initial guesses
    K = np.mean(k_grid)
    H = np.mean(h_grid)
    L = 0.3
    Z = np.mean(Z_grid)
    I_e = 0.1
    T = 0.1
    D = 0
    
    print("Starting steady state solver with Golden Section Search...")
    print(f"Initial values: K={K:.4f}, H={H:.4f}, L={L:.4f}, Z={Z:.4f}, I_e={I_e:.4f}, T={T:.4f}, D={D:.4f}")
    
    for iteration in range(max_iter):
        Y = production(K, H, L, Z)
        r = interest_rate(K, H, L, Z)
        w = wage(K, H, L, Z)
        
        V = solve_household_problem(K, H, L, Z, I_e)
        
        # Compute new aggregates using Golden Section Search
        def objective_K(K_val):
            return -np.mean([policy(K_val, h)[0] for h in h_grid])
        
        def objective_H(H_val):
            return -np.mean([H_val for _ in k_grid])
        
        def objective_L(L_val):
            return -np.mean([policy(k, h)[1] for k in k_grid for h in h_grid])
        
        K_new = golden_section_search(objective_K, k_grid[0], k_grid[-1])
        H_new = golden_section_search(objective_H, h_grid[0], h_grid[-1])
        L_new = golden_section_search(objective_L, 0, 1)
        
        D_new = government_budget(Y, K, H, L, Z, I_e, T, D, r)
        
        # Check convergence
        K_diff = abs(K_new - K)
        H_diff = abs(H_new - H)
        L_diff = abs(L_new - L)
        D_diff = abs(D_new - D)
        
        print(f"Iteration {iteration + 1}:")
        print(f"  K={K_new:.4f} (Δ={K_diff:.6f})")
        print(f"  H={H_new:.4f} (Δ={H_diff:.6f})")
        print(f"  L={L_new:.4f} (Δ={L_diff:.6f})")
        print(f"  D={D_new:.4f} (Δ={D_diff:.6f})")
        
        if K_diff < tol and H_diff < tol and L_diff < tol and D_diff < tol:
            print("Convergence achieved!")
            break
        
        # Update variables
        K, H, L, D = K_new, H_new, L_new, D_new
        
        # Adjust government policies
        I_e = max(0, min(0.1 * Y, 0.2 * (nu * Y - D)))
        Z = max(Z_grid[0], min(Z_grid[-1], Z + 0.1 * (0.2 * Y - Z)))
        T = max(0, min(0.1 * Y, 0.1 * (nu * Y - D - I_e - (Z - (1 - delta) * Z) / (1 - sigma_bar))))
        
        print(f"  Z={Z:.4f}, I_e={I_e:.4f}, T={T:.4f}")
        print()
    
    if iteration == max_iter - 1:
        print("Warning: Maximum iterations reached without convergence.")
    
    Y = production(K, H, L, Z)
    return K, H, L, Z, I_e, T, D, Y

# Solve for steady state
print("Solving for steady state...")
K_ss, H_ss, L_ss, Z_ss, I_e_ss, T_ss, D_ss, Y_ss = solve_steady_state()

print("\nFinal Steady State Results:")
print(f"Capital (K): {K_ss:.4f}")
print(f"Human Capital (H): {H_ss:.4f}")
print(f"Labor (L): {L_ss:.4f}")
print(f"Public Infrastructure (Z): {Z_ss:.4f}")
print(f"Education Investment (I_e): {I_e_ss:.4f}")
print(f"Transfers (T): {T_ss:.4f}")
print(f"Public Debt (D): {D_ss:.4f}")
print(f"Output (Y): {Y_ss:.4f}")

Solving for steady state...
Starting steady state solver with Golden Section Search...
Initial values: K=10.0000, H=5.5000, L=0.3000, Z=5.5000, I_e=0.1000, T=0.1000, D=0.0000


/var/folders/w4/8bdyf7817wzghq2ypzxtqpcm0000gn/T/ipykernel_34575/3714640005.py:33: RuntimeWarning: divide by zero encountered in scalar power
  return ((c * (1 - l)**gamma)**(1 - eta) - 1) / (1 - eta)


IndexError: index 50 is out of bounds for axis 1 with size 50